## Create GeoJSON files from US Census shapefiles
#### Census source: https://www2.census.gov/geo/tiger/TIGER2021/TRACT/

In [4]:
# dependencies
import geopandas as gpd
import zipfile
import os

In [5]:
# create geojson files from zipped shp file
for i in range(1,79):
    try:
        # format fips codes
        if i < 10:
            n = f'0{i}'
        else:
            n = i
            
        # extract file from zip folder
        path = f'../geojson/tl_2021_{n}_tract.zip'
        file_shp = f'tl_2021_{n}_tract.shp'
        file_shx = f'tl_2021_{n}_tract.shx'
        file_dbf = f'tl_2021_{n}_tract.dbf'
        file_prj = f'tl_2021_{n}_tract.prj'
            
        with zipfile.ZipFile(path, 'r') as zf:
            zf.extract(file_shp)
            zf.extract(file_shx)
            zf.extract(file_dbf)
            zf.extract(file_prj)
            
        # convert shp file to geojson
        gdf = gpd.read_file(f'../geojson/tl_2021_{n}_tract.shp')
        geojson_str = gdf.to_json()
            
        # save geojson to file
        gdf.to_file(f'tl_2021_{n}_tract.geojson', driver='GeoJSON')            
                
        # remove shp and shx files
        os.remove(file_shp)
        os.remove(file_shx)
        os.remove(file_dbf)
        os.remove(file_prj)
            
    except Exception as e:
        print(f'Error: {str(e)}, {i}')

Error: [Errno 2] No such file or directory: '../geojson/tl_2021_03_tract.zip', 3
Error: [Errno 2] No such file or directory: '../geojson/tl_2021_07_tract.zip', 7
Error: [Errno 2] No such file or directory: '../geojson/tl_2021_14_tract.zip', 14
Error: [Errno 2] No such file or directory: '../geojson/tl_2021_43_tract.zip', 43
Error: [Errno 2] No such file or directory: '../geojson/tl_2021_52_tract.zip', 52
Error: [Errno 2] No such file or directory: '../geojson/tl_2021_57_tract.zip', 57
Error: [Errno 2] No such file or directory: '../geojson/tl_2021_58_tract.zip', 58
Error: [Errno 2] No such file or directory: '../geojson/tl_2021_59_tract.zip', 59
Error: [Errno 2] No such file or directory: '../geojson/tl_2021_61_tract.zip', 61
Error: [Errno 2] No such file or directory: '../geojson/tl_2021_62_tract.zip', 62
Error: [Errno 2] No such file or directory: '../geojson/tl_2021_63_tract.zip', 63
Error: [Errno 2] No such file or directory: '../geojson/tl_2021_64_tract.zip', 64
Error: [Errno 2] N

In [ ]:
## to do - add percentages to geosjson records